In [1]:
import os
import logging
from telegram import Update, ReplyKeyboardMarkup
from telegram.ext import (
    ApplicationBuilder,
    CommandHandler,
    ContextTypes,
    MessageHandler,
    filters,
)
from dotenv import load_dotenv
import requests

load_dotenv()

TOKEN = os.getenv("TELEGRAM_TOKEN")

print("TELEGRAM_TOKEN:", TOKEN is not None)

TELEGRAM_TOKEN: True


In [2]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

In [3]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    reply_keyboard = [['Цитата'],
                      ['Анекдот'],
                      ['Имя'],
                      ['Восход и закат'],
                      ['Да или нет']]
    markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)
    await update.message.reply_text(
        "Привет! Я ваш помощник. Чем могу помочь?",
        reply_markup=markup
    )

In [4]:
async def quote(update: Update, context: ContextTypes.DEFAULT_TYPE):
    url = "https://ron-swanson-quotes.herokuapp.com/v2/quotes"
    response = requests.get(url)
    if response.status_code != 200:
        await update.message.reply_text("Не удалось получить цитату. Попробуйте позже.")
        return
    data = response.json()
    quote = data[0]
    if quote:
        await update.message.reply_text(quote)
    else:
        await update.message.reply_text("Не удалось получить цитату. Попробуйте позже.")

In [5]:
async def joke(update: Update, context: ContextTypes.DEFAULT_TYPE):
    url = "https://official-joke-api.appspot.com/random_joke"
    response = requests.get(url)
    if response.status_code != 200:
        await update.message.reply_text("Не удалось получить анекдот. Попробуйте позже.")
        return
    data = response.json()
    setup = data.get('setup')
    punchline = data.get('punchline')
    if setup and punchline:
        await update.message.reply_text(f"{setup}\n\n{punchline}")
    else:
        await update.message.reply_text("Не удалось получить анекдот. Попробуйте позже.")

In [6]:
async def name(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if context.args:
        name = ' '.join(context.args)
        url = f'https://api.genderize.io?name={name}'
        response = requests.get(url)
        data = response.json()
        gender = data.get('gender')
        gender_dict = {'male': 'мужчина', 'female': 'девушка'}
        probability = data.get('probability')
        if gender and probability:
            await update.message.reply_text(f'С вероятностью в {round(probability * 100, 2)}% уверен, что Вы {gender_dict[gender]}')
        else:
            await update.message.reply_text("Ваше имя не распознать.")
    else:
        await update.message.reply_text("Пожалуйста, введите ваше имя на английском. Пример: /name Mary")

In [7]:
async def sun(update: Update, context: ContextTypes.DEFAULT_TYPE):
    url = "https://api.sunrise-sunset.org/json?lat=36.7201600&lng=-4.4203400"
    response = requests.get(url)
    if response.status_code != 200:
        await update.message.reply_text("Не удалось получить информацию. Попробуйте позже.")
        return
    data = response.json()
    results = data.get('results')
    sunrise = results.get('sunrise')
    sunset = results.get('sunset')
    if sunrise and sunset:
        await update.message.reply_text(f"Время восхода: {sunrise}\nВремя заката: {sunset}")
    else:
        await update.message.reply_text("Не удалось получить информацию. Попробуйте позже.")

In [8]:
async def yesno(update: Update, context: ContextTypes.DEFAULT_TYPE):
    url = "https://yesno.wtf/api"
    response = requests.get(url)
    if response.status_code != 200:
        await update.message.reply_text("Не удалось получить ответ. Попробуйте позже.")
        return
    data = response.json()
    answer = data.get('answer')
    answer_dict = {'yes': 'Да', 'no': 'Нет'}
    image = data.get('image')
    if answer:
        await update.message.reply_text(answer_dict[answer])
        if image:
            await update.message.reply_photo(photo=image)
    else:
        await update.message.reply_text("Не удалось получить ответ. Попробуйте позже.")

In [9]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text.lower()
    if text == 'цитата':
        await quote(update, context)
    elif text == 'анекдот':
        await joke(update, context)
    elif text == 'имя':
        await update.message.reply_text("Введите ваше имя /name <имя>")
    elif text == 'восход и закат':
        await sun(update, context)
    elif text == 'да или нет':
        await yesno(update, context)
    else:
        await update.message.reply_text("Извините, я не понимаю эту команду.")

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from telegram.ext import CommandHandler, MessageHandler, filters

async def main():
    application = ApplicationBuilder().token(TOKEN).build()

    application.add_handler(CommandHandler('start', start))
    application.add_handler(CommandHandler('quote', quote))
    application.add_handler(CommandHandler('joke', joke))
    application.add_handler(CommandHandler('name', name))
    application.add_handler(CommandHandler('sun', sun))
    application.add_handler(CommandHandler('yesno', yesno))

    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    await application.run_polling()

In [12]:
import asyncio

loop = asyncio.get_event_loop()
loop.create_task(main())

print("Бот запущен. Теперь вы можете взаимодействовать с ним через Telegram.")

Бот запущен. Теперь вы можете взаимодействовать с ним через Telegram.
